In [32]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [33]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_1_Radiotherapy-Treatment/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-17 16:17:55.857628


## Step 1 - Generate Mathematical Formulation 

In [34]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [35]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [36]:
Markdown(response_1_text)

To formulate an integer linear optimization model for this problem, we will define the following parameters, decision variables, objective function, and constraints:

Parameters:
- n: number of patients (n = 17)
- C: total maximal proton fraction capacity (C = 100)
- BED_ij: BED score for patient i when using j proton fractions (given data)

Decision Variables:
- x_ij: number of proton fractions assigned to patient i (i = 1, ..., n; j = 0, ..., 15)
  - x_ij is an integer variable
  - x_ij ∈ {0, 1} for each i and j (binary variable)

Objective Function:
Let's denote the total BED score as Z. We want to maximize this value:

Z = ∑ ∑ BED_ij * x_ij
   i=1 j=0

Constraints:
1. Each patient receives exactly 15 fractions (a mix of proton and photon fractions):

∑ x_ij = 15 for every patient i (i = 1, ..., n)
  j=0

2. The total number of proton fractions used does not exceed the capacity C:

∑ ∑ j * x_ij ≤ C
   i=1 j=0

3. A patient can only receive a single number of proton fractions (either 0, 1, ..., or 15):

∑ x_ij = 1 for every patient i (i = 1, ..., n)
  j=0

This formulation provides an integer linear optimization model that can be used to determine the optimal number of proton fractions for each patient while maximizing the total BED score. By solving this model, we can determine the best treatment plan for the given patients under the proton fraction capacity limit.

In [37]:
print(response_1_text)

To formulate an integer linear optimization model for this problem, we will define the following parameters, decision variables, objective function, and constraints:

Parameters:
- n: number of patients (n = 17)
- C: total maximal proton fraction capacity (C = 100)
- BED_ij: BED score for patient i when using j proton fractions (given data)

Decision Variables:
- x_ij: number of proton fractions assigned to patient i (i = 1, ..., n; j = 0, ..., 15)
  - x_ij is an integer variable
  - x_ij ∈ {0, 1} for each i and j (binary variable)

Objective Function:
Let's denote the total BED score as Z. We want to maximize this value:

Z = ∑ ∑ BED_ij * x_ij
   i=1 j=0

Constraints:
1. Each patient receives exactly 15 fractions (a mix of proton and photon fractions):

∑ x_ij = 15 for every patient i (i = 1, ..., n)
  j=0

2. The total number of proton fractions used does not exceed the capacity C:

∑ ∑ j * x_ij ≤ C
   i=1 j=0

3. A patient can only receive a single number of proton fractions (either

## Step 2 - Generate the Pyomo Code

In [38]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [39]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [40]:
Markdown(response_2_text)

Here is a Python code using Pyomo to solve the optimization problem you described:

```python
import pyomo.environ as pyo
import numpy as np

# Sample data
n = 17  # number of patients
C = 100  # total maximal proton fraction capacity
BED_ij = np.random.rand(n, 16)  # BED score for patient i when using j proton fractions (sample data)

model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)
model.J = pyo.RangeSet(0, 15)

# Parameters
model.C = pyo.Param(initialize=C)
model.BED_ij = pyo.Param(model.I, model.J, initialize=BED_ij)

# Decision Variables
model.x_ij = pyo.Var(model.I, model.J, domain=pyo.Binary)

# Objective Function
def obj_function(model):
    return sum(sum(model.BED_ij[i, j] * model.x_ij[i, j] for j in model.J) for i in model.I)

model.obj = pyo.Objective(rule=obj_function, sense=pyo.maximize)

# Constraints
def constraint1(model, i):
    return sum(model.x_ij[i, j] for j in model.J) == 15

model.constraint1 = pyo.Constraint(model.I, rule=constraint1)

def constraint2(model):
    return sum(sum(j * model.x_ij[i, j] for j in model.J) for i in model.I) <= model.C

model.constraint2 = pyo.Constraint(rule=constraint2)

def constraint3(model, i):
    return sum(model.x_ij[i, j] for j in model.J) == 1

model.constraint3 = pyo.Constraint(model.I, rule=constraint3)

# Solve
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print results
print("Optimal Solution Found")
print("Objective Value:", pyo.value(model.obj))
for i in model.I:
    for j in model.J:
        if pyo.value(model.x_ij[i, j]) > 0.5:
            print(f"Patient {i} receives {j} proton fractions")
```

This code creates a Pyomo ConcreteModel and defines the parameters, decision variables, objective function, and constraints as described. It then uses the GLPK solver to find the optimal solution. The results are printed to the console, showing the objective value and the number of proton fractions assigned to each patient.

In [41]:
print(response_2_text)

Here is a Python code using Pyomo to solve the optimization problem you described:

```python
import pyomo.environ as pyo
import numpy as np

# Sample data
n = 17  # number of patients
C = 100  # total maximal proton fraction capacity
BED_ij = np.random.rand(n, 16)  # BED score for patient i when using j proton fractions (sample data)

model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)
model.J = pyo.RangeSet(0, 15)

# Parameters
model.C = pyo.Param(initialize=C)
model.BED_ij = pyo.Param(model.I, model.J, initialize=BED_ij)

# Decision Variables
model.x_ij = pyo.Var(model.I, model.J, domain=pyo.Binary)

# Objective Function
def obj_function(model):
    return sum(sum(model.BED_ij[i, j] * model.x_ij[i, j] for j in model.J) for i in model.I)

model.obj = pyo.Objective(rule=obj_function, sense=pyo.maximize)

# Constraints
def constraint1(model, i):
    return sum(model.x_ij[i, j] for j in model.J) == 15

model.constraint1 = pyo.Constraint(model.I, rule=constraint1)

def const

### Code Executability

In [42]:
import pyomo.environ as pyo
import numpy as np

# Sample data
n = 17  # number of patients
C = 100  # total maximal proton fraction capacity
BED_ij = np.random.rand(n, 16)  # BED score for patient i when using j proton fractions (sample data)

model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)
model.J = pyo.RangeSet(0, 15)

# Parameters
model.C = pyo.Param(initialize=C)
model.BED_ij = pyo.Param(model.I, model.J, initialize=BED_ij)

# Decision Variables
model.x_ij = pyo.Var(model.I, model.J, domain=pyo.Binary)

# Objective Function
def obj_function(model):
    return sum(sum(model.BED_ij[i, j] * model.x_ij[i, j] for j in model.J) for i in model.I)

model.obj = pyo.Objective(rule=obj_function, sense=pyo.maximize)

# Constraints
def constraint1(model, i):
    return sum(model.x_ij[i, j] for j in model.J) == 15

model.constraint1 = pyo.Constraint(model.I, rule=constraint1)

def constraint2(model):
    return sum(sum(j * model.x_ij[i, j] for j in model.J) for i in model.I) <= model.C

model.constraint2 = pyo.Constraint(rule=constraint2)

def constraint3(model, i):
    return sum(model.x_ij[i, j] for j in model.J) == 1

model.constraint3 = pyo.Constraint(model.I, rule=constraint3)

# Solve
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print results
print("Optimal Solution Found")
print("Objective Value:", pyo.value(model.obj))
for i in model.I:
    for j in model.J:
        if pyo.value(model.x_ij[i, j]) > 0.5:
            print(f"Patient {i} receives {j} proton fractions")

2024-05-17 16:26:00,100 ERROR pyomo.core: Rule failed for Param 'BED_ij' with index 0:
KeyError: "Index '0' is not valid for indexed component 'BED_ij'"
2024-05-17 16:26:00,102 ERROR pyomo.core: Constructing component 'BED_ij' from data=None failed:
KeyError: "Index '0' is not valid for indexed component 'BED_ij'"


KeyError: "Index '0' is not valid for indexed component 'BED_ij'"

### Solution Correctness

In [51]:
import pyomo.environ as pyo
import numpy as np

# Sample data
n = 17  # number of patients
C = 100  # total maximal proton fraction capacity
BED = {}
with open(FILE_PATH_PROBLEM + 'ProblemData.csv', 'r') as f:
    for i, line in enumerate(f):
        for j, val in enumerate(line.strip().split(',')):
            BED[(i + 1, j)] = float(val)
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, n)
model.J = pyo.RangeSet(0, 15)

# Parameters
model.C = pyo.Param(initialize=C)

# Decision Variables
model.x_ij = pyo.Var(model.I, model.J, domain=pyo.Binary)

# Objective Function
def obj_function(model):
    return sum(sum(BED[i, j] * model.x_ij[i, j] for j in model.J) for i in model.I)

model.obj = pyo.Objective(rule=obj_function, sense=pyo.maximize)

# Constraints
def constraint1(model, i):
    return sum(model.x_ij[i, j] for j in model.J) == 15

model.constraint1 = pyo.Constraint(model.I, rule=constraint1)

def constraint2(model):
    return sum(sum(j * model.x_ij[i, j] for j in model.J) for i in model.I) <= model.C

model.constraint2 = pyo.Constraint(rule=constraint2)

def constraint3(model, i):
    return sum(model.x_ij[i, j] for j in model.J) == 1

model.constraint3 = pyo.Constraint(model.I, rule=constraint3)

# Solve
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print results
print("Optimal Solution Found")
print(results)

Optimal Solution Found

Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 36
  Number of variables: 273
  Number of nonzeros: 800
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.07588005065917969

